In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import time
import re

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [2]:
# URL del artículo en Wayback Machine
url = "https://web.archive.org/web/20180304015632/https://www.elespectador.com/opinion/independencia-en-ceros-columna-742192"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Diccionario para meses en español
meses = {
    "Ene": "enero", "Feb": "febrero", "Mar": "marzo", "Abr": "abril",
    "May": "mayo", "Jun": "junio", "Jul": "julio", "Ago": "agosto",
    "Sep": "septiembre", "Oct": "octubre", "Nov": "noviembre", "Dic": "diciembre"
}

# 1. Extraer título
titulo = soup.find('h1').get_text(strip=True)

# 2. Extraer fecha formateada
fecha_element = soup.find('div', class_='node-post-date')
if fecha_element:
    fecha_texto = fecha_element.get_text(strip=True).split(' - ')[0]
    dia, mes_abrev, anio = fecha_texto.split()
    fecha_formateada = f"{dia} de {meses[mes_abrev]} de {anio}"
else:
    fecha_formateada = "Fecha no encontrada"

# 3. Extraer autor (texto después de "Por:")
autor_element = soup.find('span', class_='by')  # Localiza el span con "Por:"
if autor_element:
    autor = autor_element.next_sibling.strip()  # Toma el texto HERMANO siguiente al span
else:
    autor = "Autor no encontrado"

# 4. Extraer contenido limpio con párrafos (versión mejorada)
contenido_div = soup.find('div', class_='node-body')
if contenido_div:
    # Primero eliminar el div no deseado si existe
    info_node = contenido_div.find('div', class_='info_node_hide')
    if info_node:
        info_node.decompose()  # Esto elimina completamente el div y su contenido

    # Eliminar solo elementos no deseados (scripts, iframes, etc.)
    for element in contenido_div(['script', 'style', 'iframe', 'img', 'figure']):
        element.decompose()

    # Procesar cada párrafo conservando formato semántico
    parrafos = []
    for p in contenido_div.find_all('p'):
        # Extraer todo el texto del párrafo incluyendo etiquetas de formato
        texto_parrafo = p.get_text(' ', strip=True)  # El espacio une elementos separados
        if texto_parrafo:
            # Limpieza final de espacios múltiples
            texto_parrafo = ' '.join(texto_parrafo.split())
            parrafos.append(texto_parrafo)

    contenido = '\n\n'.join(parrafos)
else:
    contenido = "Contenido no encontrado"

# Resultados
print(f"Título: {titulo}")
print(f"Fecha: {fecha_formateada}")
print(f"Autor: {autor}")
print("\n--- CONTENIDO ---\n")
print(contenido)

Título: Independencia en ceros
Fecha: 2 de marzo de 2018
Autor: José Roberto Acosta

--- CONTENIDO ---

Hace una semana para nadie era un problema los tres ceros en nuestros billetes, pero para ocultar su inoperancia y conflicto de intereses en el caso Odebrecht, el fiscal general Néstor Humberto Martinez salió con tan costosa cortina de humo.

Es sospechoso que, siendo legal y necesaria la independencia del fiscal general de la Nación respecto al Poder Ejecutivo del Estado, de manera simultánea saliera el ministro de Hacienda a decir que ya tenía listo el proyecto de ley parta tramitarlo en el Congreso generándole un gasto de por lo menos $400.000 millones a la nación, sin contar los costos en cambios y software y sistemas de contabilidad para las empresas y las millonadas que derrochará el Gobierno en campañas publicitarias de pedagogía por los tres años que duraría la transición al “nuevo peso”, tiempo suficiente para la operación de lavado de las caletas mencionadas por el fiscal.


In [5]:
# Usamos with para que el archivo se cierre automáticamente
with open("urls.txt", "r") as f:
    # Leemos todas las líneas y filtramos las vacías
    urls = [line.strip() for line in f if line.strip()]

# Eliminamos duplicados
urls = list(dict.fromkeys(urls))

# Diccionario para meses en español
meses = {
    "Ene": "enero", "Feb": "febrero", "Mar": "marzo", "Abr": "abril",
    "May": "mayo", "Jun": "junio", "Jul": "julio", "Ago": "agosto",
    "Sep": "septiembre", "Oct": "octubre", "Nov": "noviembre", "Dic": "diciembre"
}

# Lista para almacenar todos los resultados
datos = []
n = 0
for url in urls:
    try:
        print(f"Procesando: {url}")
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 1. Extraer título
        titulo = soup.find('h1').get_text(strip=True) if soup.find('h1') else "Título no encontrado"

        # 2. Extraer fecha formateada
        fecha_element = soup.find('div', class_='node-post-date')
        if fecha_element:
            fecha_texto = fecha_element.get_text(strip=True).split(' - ')[0]
            try:
                dia, mes_abrev, anio = fecha_texto.split()
                fecha_formateada = f"{dia} de {meses[mes_abrev]} de {anio}"
            except:
                fecha_formateada = fecha_texto
        else:
            fecha_formateada = "Fecha no encontrada"

        # 3. Extraer autor
        autor_element = soup.find('span', class_='by')
        if autor_element:
            autor = autor_element.next_sibling.strip() if autor_element.next_sibling else "Autor no encontrado"
        else:
            autor = "Autor no encontrado"

        # 4. Extraer contenido limpio con párrafos (versión mejorada)
        contenido_div = soup.find('div', class_='node-body')
        if contenido_div:
            # Primero eliminar el div no deseado si existe
            info_node = contenido_div.find('div', class_='info_node_hide')
            if info_node:
                info_node.decompose()  # Esto elimina completamente el div y su contenido

            # Eliminar solo elementos no deseados (scripts, iframes, etc.)
            for element in contenido_div(['script', 'style', 'iframe', 'img', 'figure']):
                element.decompose()

            # Procesar cada párrafo conservando formato semántico
            parrafos = []
            for p in contenido_div.find_all('p'):
                # Extraer todo el texto del párrafo incluyendo etiquetas de formato
                texto_parrafo = p.get_text(' ', strip=True)  # El espacio une elementos separados
                if texto_parrafo:
                    # Limpieza final de espacios múltiples
                    texto_parrafo = ' '.join(texto_parrafo.split())
                    parrafos.append(texto_parrafo)

            contenido = '\n\n'.join(parrafos)
        else:
            contenido = "Contenido no encontrado"

        # Agregar a la lista de datos
        datos.append({
            'Autor': autor,
            'Fecha': fecha_formateada,
            'Título': titulo,
            'Contenido': contenido,
            'URL': url
        })
        #time.sleep(1)
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        datos.append({
            'Autor': f"Error: {str(e)}",
            'Fecha': "",
            'Título': "",
            'Contenido': "",
            'URL': url
        })
    n = n + 1
    if n % 20 == 0 and n < len(urls):
        print(f"Procesados: {n} de {len(urls)}")
        time.sleep(150)



# Crear DataFrame y guardar como CSV
df = pd.DataFrame(datos)

# Ordenar columnas
column_order = ['Autor', 'Fecha', 'Título', 'Contenido', 'URL']
df = df[column_order]

# Crear archivo Excel
nombre_archivo = "resultados_articulos.xlsx"
with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Artículos')

    # Ajustar el ancho de las columnas
    worksheet = writer.sheets['Artículos']
    worksheet.column_dimensions['A'].width = 25  # Autor
    worksheet.column_dimensions['B'].width = 20  # Fecha
    worksheet.column_dimensions['C'].width = 40  # Título
    worksheet.column_dimensions['D'].width = 80  # Contenido
    worksheet.column_dimensions['E'].width = 60  # URL

print(f"\nProceso completado. Resultados guardados en {nombre_archivo}")

Procesando: https://web.archive.org/web/20180701051737/https://www.elespectador.com/opinion/matar-un-pajarito-columna-797549
Procesando: https://web.archive.org/web/20180701051737/https://www.elespectador.com/opinion/el-rey-de-la-contratacion-columna-797558
Procesando: https://web.archive.org/web/20180701051737/https://www.elespectador.com/opinion/problemas-de-identidad-columna-797551
Procesando: https://web.archive.org/web/20180701051737/https://www.elespectador.com/opinion/de-la-historia-la-memoria-columna-797489
Procesando: https://web.archive.org/web/20180701051737/https://www.elespectador.com/opinion/pesar-de-ustedes-columna-797556
Procesando: https://web.archive.org/web/20180701051737/https://www.elespectador.com/opinion/un-paso-adelante-dos-atras-columna-797550
Procesando: https://web.archive.org/web/20180701051737/https://www.elespectador.com/opinion/bienvenida-fiscal-bensouda-columna-797552
Procesando: https://web.archive.org/web/20180701051737/https://www.elespectador.com/opi

# Para extraer links

In [34]:
import requests
from bs4 import BeautifulSoup

fecha = "20200616194558"
url = "https://web.archive.org/web/" + fecha + "/https://www.elespectador.com/opinion/"
inicio = "/web/" + fecha + "mp_/https://www.elespectador.com/"

try:
    response = requests.get(url)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error al hacer la petición HTTP: {e}")
    exit()

soup = BeautifulSoup(response.text, 'html.parser')

column_items = soup.find_all('li', class_='views-row')

column_links = []
base_url = "https://web.archive.org"

excluded_authors = [
    "Columnista invitado EE",
    "Cartas de los lectores",
    "Las igualadas",
    "La Pulla",
    "Antieditorial",
    "Columna del lector",
    "La Puesverdad"
]

for item in column_items:
    # Obtener el autor de manera más robusta
    author = ""
    author_div = item.find('div', class_='views-field-field-columnist')
    if author_div:
        author_link = author_div.find('a')
        if author_link:
            author = author_link.get_text(strip=True)
        else:
            author = author_div.get_text(strip=True).replace("Por: ", "")
    
    # Verificar si el autor está en la lista de exclusiones
    if any(excluded.lower() in author.lower() for excluded in excluded_authors):
        continue
    
    # Obtener el enlace de manera más segura
    title_div = item.find('div', class_='views-field-title')
    if not title_div:
        continue
        
    title_link = title_div.find('a')
    if not title_link or 'href' not in title_link.attrs:
        continue
    
    href = title_link['href']
    
    # Verificar que sea un enlace de columna válido

    #if (href.startswith(inicio) and 'columna-' in href):
    if (href.startswith(inicio) in href):
        full_url = base_url + href if not href.startswith(base_url) else href
        column_links.append(full_url)

# Eliminar duplicados manteniendo el orden
seen = set()
column_links = [x for x in column_links if not (x in seen or seen.add(x))]

print(f"\nSe encontraron {len(column_links)} enlaces de columnas válidos:")
for i, link in enumerate(column_links, 1):
    print(f"{link}")




Se encontraron 0 enlaces de columnas válidos:


In [38]:
import requests
import json
from bs4 import BeautifulSoup
import re

url = "https://web.archive.org/web/20200618142303/https://www.elespectador.com/opinion/"

try:
    response = requests.get(url)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error al hacer la petición HTTP: {e}")
    exit()

soup = BeautifulSoup(response.text, 'html.parser')

# Buscar el script que contiene los datos JSON
script_tag = soup.find('script', type='application/javascript')
if not script_tag:
    print("No se encontró el script con los datos JSON")
    exit()

# Extraer el JSON del script
json_data = {}
try:
    # Buscar la variable Fusion.globalContent
    match = re.search(r'Fusion\.globalContent\s*=\s*({.*?});', script_tag.string, re.DOTALL)
    if match:
        json_str = match.group(1)
        json_data = json.loads(json_str)
except (AttributeError, json.JSONDecodeError) as e:
    print(f"Error al procesar los datos JSON: {e}")
    exit()

# Lista para almacenar los enlaces válidos
column_links = []
base_url = "https://web.archive.org"

# Excluir estos tipos de contenido
excluded_types = [
    "Columnista invitado EE",
    "Cartas de los lectores",
    "Las igualadas",
    "La Pulla",
    "Antieditorial",
    "Columna del lector",
    "La Puesverdad"
]

# Procesar los elementos de contenido
if 'content_elements' in json_data:
    for element in json_data['content_elements']:
        if element.get('type') == 'story':
            # Obtener URL canónica
            canonical_url = element.get('canonical_url', '')
            
            # Verificar que sea una columna de opinión
            if canonical_url and '/opinion/' in canonical_url:
                # Construir URL de Wayback Machine
                wayback_date = "20200618142303"  # Puedes extraer esto de la URL original si es variable
                wayback_url = f"{base_url}/web/{wayback_date}mp_/{canonical_url}"
                
                # Verificar que no sea de los tipos excluidos
                title = element.get('headlines', {}).get('basic', '').lower()
                if not any(excluded.lower() in title for excluded in excluded_types):
                    column_links.append(wayback_url)

# Eliminar duplicados
column_links = list(dict.fromkeys(column_links))

# Mostrar resultados
print(f"\nSe encontraron {len(column_links)} enlaces de columnas válidos:")
for i, link in enumerate(column_links, 1):
    print(f"{i}. {link}")



Se encontraron 0 enlaces de columnas válidos:
